In [2]:
%config Completer.use_jedi = False

In [3]:
from bravado.client import SwaggerClient

cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/api-docs',
                                config={"validate_requests":False,"validate_responses":False})
print(cbioportal)

SwaggerClient(https://www.cbioportal.org/api)


In [41]:
import pandas as pd

In [100]:
study_id='skcm_tcga_pan_can_atlas_2018'

In [101]:
def get_pandas_df(list):
    return pd.DataFrame.from_dict([
            dict(
                {k:getattr(m,k) for k in m})
                    for m in list
                    ])

In [102]:
mol_profiles=cbioportal.Molecular_Profiles.getAllMolecularProfilesInStudyUsingGET(studyId=study_id).result()
mpdf = get_pandas_df(mol_profiles)
mpdf.head()

,datatype,description,genericAssayType,molecularAlterationType,molecularProfileId,name,pivotThreshold,showProfileInAnalysisTab,sortOrder,study,studyId
0,LOG2-VALUE,Protein expression measured by reverse-phase p...,None,PROTEIN_LEVEL,skcm_tcga_pan_can_atlas_2018_rppa,Protein expression (RPPA),NaN,False,None,None,skcm_tcga_pan_can_atlas_2018
1,Z-SCORE,"Protein expression, measured by reverse-phase ...",None,PROTEIN_LEVEL,skcm_tcga_pan_can_atlas_2018_rppa_Zscores,Protein expression z-scores (RPPA),NaN,True,None,None,skcm_tcga_pan_can_atlas_2018
2,DISCRETE,Putative copy-number from GISTIC 2.0. Values: ...,None,COPY_NUMBER_ALTERATION,skcm_tcga_pan_can_atlas_2018_gistic,Putative copy-number alterations from GISTIC,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018
3,CONTINUOUS,"mRNA Expression, RSEM (Batch normalized from I...",None,MRNA_EXPRESSION,skcm_tcga_pan_can_atlas_2018_rna_seq_v2_mrna,"mRNA Expression, RSEM (Batch normalized from I...",NaN,False,None,None,skcm_tcga_pan_can_atlas_2018
4,Z-SCORE,mRNA expression z-scores (RNA Seq V2 RSEM) com...,None,MRNA_EXPRESSION,skcm_tcga_pan_can_atlas_2018_rna_seq_v2_mrna_m...,mRNA expression z-scores relative to diploid s...,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018


In [31]:
mol_profile_id = mpdf.loc[mpdf['name'] == 'Mutations']['molecularProfileId'].item()
mol_profile_id

'skcm_tcga_pan_can_atlas_2018_mutations'

In [78]:
genes = cbioportal.Genes.getAllGenesUsingGET(keyword='BRAF').result()
gdf = get_pandas_df(genes)
gdf.head()

,entrezGeneId,geneticEntityId,hugoGeneSymbol,type
0,673,None,BRAF,protein-coding
1,286494,None,BRAFP1,pseudo
2,-11087,None,BRAF_PS147,phosphoprotein
3,-11088,None,BRAF_PS151,phosphoprotein
4,-59030,None,BRAF_PS314,phosphoprotein


In [45]:
entrez_gene_id = gdf.loc[gdf['hugoGeneSymbol'] == 'BRAF']['entrezGeneId'].item()
entrez_gene_id

673

In [79]:
mutations = cbioportal.Mutations.getMutationsInMolecularProfileBySampleListIdUsingGET(
    sampleListId='skcm_tcga_pan_can_atlas_2018_cnaseq', 
    molecularProfileId=mol_profile_id,
    entrezGeneId=entrez_gene_id).result()

mdf = get_pandas_df(mutations)
mdf.columns

Index(['alleleSpecificCopyNumber', 'aminoAcidChange', 'center', 'chr',
       'driverFilter', 'driverFilterAnnotation', 'driverTiersFilter',
       'driverTiersFilterAnnotation', 'endPosition', 'entrezGeneId',
       'fisValue', 'functionalImpactScore', 'gene', 'keyword', 'linkMsa',
       'linkPdb', 'linkXvar', 'molecularProfileId', 'mutationStatus',
       'mutationType', 'ncbiBuild', 'normalAltCount', 'normalRefCount',
       'patientId', 'proteinChange', 'proteinPosEnd', 'proteinPosStart',
       'referenceAllele', 'refseqMrnaId', 'sampleId', 'startPosition',
       'studyId', 'tumorAltCount', 'tumorRefCount', 'uniquePatientKey',
       'uniqueSampleKey', 'validationStatus', 'variantAllele', 'variantType'],
      dtype='object')

In [88]:
all_patients = mdf['patientId'].tolist()
len(set(all_patients))

191

In [81]:
altered_mdf = mdf[mdf['proteinChange']=='V600E']
altered_mdf.head()

,alleleSpecificCopyNumber,aminoAcidChange,center,chr,driverFilter,driverFilterAnnotation,driverTiersFilter,driverTiersFilterAnnotation,endPosition,entrezGeneId,...,sampleId,startPosition,studyId,tumorAltCount,tumorRefCount,uniquePatientKey,uniqueSampleKey,validationStatus,variantAllele,variantType
8,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-BF-AAP0-06,140453136,skcm_tcga_pan_can_atlas_2018,54,64,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1CRi1BQVAwLTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
9,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q4-06,140453136,skcm_tcga_pan_can_atlas_2018,26,29,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE0LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
10,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q5-06,140453136,skcm_tcga_pan_can_atlas_2018,33,17,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE1LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
11,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q6-06,140453136,skcm_tcga_pan_can_atlas_2018,17,36,VENHQS1EMy1BMVE2OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE2LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
12,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q7-06,140453136,skcm_tcga_pan_can_atlas_2018,4,33,VENHQS1EMy1BMVE3OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE3LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP


In [89]:
altered_patients = altered_mdf['patientId'].tolist()
len(set(altered_patients))

152

In [85]:
unaltered_patients = list(set(all_patients) - set(altered_patients))
len(unaltered_patients)

39

In [105]:
attribute_ids = ["OS_STATUS","OS_MONTHS","DFS_STATUS","DFS_MONTHS","DSS_STATUS","DSS_MONTHS","PFS_STATUS","PFS_MONTHS"]

clinical_data_single_study_filter = {
  'attributeIds': attribute_ids,
  'ids': altered_patients
}

altered_clinical = cbioportal.Clinical_Data.fetchAllClinicalDataInStudyUsingPOST(studyId = studyId, 
                                                              clinicalDataType='PATIENT',
                                                              clinicalDataSingleStudyFilter=clinical_data_single_study_filter).result()
altered_clinical_df = get_pandas_df(altered_clinical)
altered_clinical_df.head()

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
0,None,DSS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999
1,None,DSS_STATUS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:ALIVE OR DEAD TUMOR FREE
2,None,OS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999
3,None,OS_STATUS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
4,None,PFS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999


In [106]:
clinical_data_single_study_filter = {
  'attributeIds': attribute_ids,
  'ids': unaltered_patients
}

unaltered_clinical = cbioportal.Clinical_Data.fetchAllClinicalDataInStudyUsingPOST(studyId = studyId, 
                                                              clinicalDataType='PATIENT',
                                                              clinicalDataSingleStudyFilter=clinical_data_single_study_filter).result()
unaltered_clinical_df = get_pandas_df(unaltered_clinical)
unaltered_clinical_df.head()

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
0,None,DSS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.029950357
1,None,DSS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DEAD WITH TUMOR
2,None,OS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.029950357
3,None,OS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DECEASED
4,None,PFS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.005325969000001


In [11]:
cbioportal.Samples.getAllSamplesInStudyUsingGET(studyId='skcm_tcga_pan_can_atlas_2018').result()

[Sample(copyNumberSegmentPresent=None, patientId='TCGA-3N-A9WB', profiledForFusions=None, sampleId='TCGA-3N-A9WB-06', sampleType='Metastatic', sequenced=None, studyId='skcm_tcga_pan_can_atlas_2018', uniquePatientKey='VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bGFzXzIwMTg', uniqueSampleKey='VENHQS0zTi1BOVdCLTA2OnNrY21fdGNnYV9wYW5fY2FuX2F0bGFzXzIwMTg'),
 Sample(copyNumberSegmentPresent=None, patientId='TCGA-3N-A9WC', profiledForFusions=None, sampleId='TCGA-3N-A9WC-06', sampleType='Metastatic', sequenced=None, studyId='skcm_tcga_pan_can_atlas_2018', uniquePatientKey='VENHQS0zTi1BOVdDOnNrY21fdGNnYV9wYW5fY2FuX2F0bGFzXzIwMTg', uniqueSampleKey='VENHQS0zTi1BOVdDLTA2OnNrY21fdGNnYV9wYW5fY2FuX2F0bGFzXzIwMTg'),
 Sample(copyNumberSegmentPresent=None, patientId='TCGA-3N-A9WD', profiledForFusions=None, sampleId='TCGA-3N-A9WD-06', sampleType='Metastatic', sequenced=None, studyId='skcm_tcga_pan_can_atlas_2018', uniquePatientKey='VENHQS0zTi1BOVdEOnNrY21fdGNnYV9wYW5fY2FuX2F0bGFzXzIwMTg', uniqueSampleKey